In [1]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

In [2]:
data = pd.read_csv("bsc_project_set.csv", index_col=0)
print(data.shape)
print(data.columns)
data

(3941, 27)
Index(['id', 'mort_28', 'age', 'sex', 'weight', 'height', 'pf_ratio', 'po2',
       'pco2', 'ph', 'driving_pressure', 'lung_compliance', 'map', 'bilirubin',
       'creatinine', 'platelets', 'urea', 'fio2', 'hco3', 'heart_rate',
       'minute_volume', 'peep', 'plateau_pressure', 'respiratory_rate',
       'syst_blood_pressure', 'diastolic_blood_pressure', 'peep_regime'],
      dtype='object')


,id,mort_28,age,sex,weight,height,pf_ratio,po2,pco2,ph,...,fio2,hco3,heart_rate,minute_volume,peep,plateau_pressure,respiratory_rate,syst_blood_pressure,diastolic_blood_pressure,peep_regime
0,32128372,False,75,F,93.0,168.0,241.944444,130.666667,28.111111,7.245556,...,50.000000,12.000000,115.408163,8.241111,8.777778,24.333333,18.943662,100.600000,68.200000,low
1,34100191,False,47,M,127.0,183.0,166.356061,113.076923,34.461538,7.196923,...,68.500000,12.750000,115.407407,16.628571,7.375000,17.500000,19.857143,99.000000,68.000000,low
2,38292466,False,85,M,77.5,170.0,224.000000,112.000000,38.000000,7.440000,...,51.818182,23.000000,70.041667,9.433333,5.000000,22.333333,15.833333,102.000000,46.333333,low
3,32743332,False,85,M,90.7,170.0,205.714286,72.000000,38.000000,7.500000,...,36.875000,27.000000,70.480000,7.478571,5.000000,24.000000,18.200000,108.083333,46.208333,low
4,35009126,True,64,M,70.0,183.0,173.777778,94.666667,38.666667,7.400000,...,52.500000,21.666667,73.955556,7.050000,5.428571,14.328571,15.016667,103.415094,57.377358,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3936,32959861,True,78,F,57.0,160.0,157.178571,136.833333,38.083333,7.370833,...,65.454545,22.000000,95.785714,8.511111,6.800000,22.000000,16.677419,NaN,NaN,low
3937,32761676,False,75,M,101.0,178.0,179.909091,91.714286,39.000000,7.310000,...,52.142857,17.600000,60.394737,7.250000,5.000000,15.400000,16.795455,NaN,NaN,low
3938,34058245,True,67,F,100.0,165.0,158.331733,104.250000,43.250000,7.292500,...,77.142857,18.800000,122.041667,8.813333,16.700000,30.333333,25.290323,101.789474,59.631579,high
3939,34670930,False,68,M,94.7,163.0,192.500000,90.600000,40.400000,7.414000,...,60.000000,23.500000,65.807692,8.816667,9.166667,21.285714,21.531250,158.000000,83.500000,low


In [3]:
data.query("pf_ratio<200")

,id,mort_28,age,sex,weight,height,pf_ratio,po2,pco2,ph,...,fio2,hco3,heart_rate,minute_volume,peep,plateau_pressure,respiratory_rate,syst_blood_pressure,diastolic_blood_pressure,peep_regime
1,34100191,False,47,M,127.0,183.0,166.356061,113.076923,34.461538,7.196923,...,68.500000,12.750000,115.407407,16.628571,7.375000,17.500000,19.857143,99.000000,68.000000,low
4,35009126,True,64,M,70.0,183.0,173.777778,94.666667,38.666667,7.400000,...,52.500000,21.666667,73.955556,7.050000,5.428571,14.328571,15.016667,103.415094,57.377358,low
8,37153661,True,82,F,46.3,157.0,140.071429,109.000000,36.000000,7.340000,...,60.000000,17.500000,96.090909,7.962500,6.250000,19.000000,21.292683,100.000000,46.235294,low
11,30757476,False,72,M,107.0,178.0,191.250000,87.500000,49.000000,7.315000,...,39.285714,25.333333,58.307692,11.275000,8.000000,19.571429,18.757576,109.000000,63.333333,low
15,39625056,True,78,M,90.0,180.0,129.076389,98.571429,39.571429,7.234286,...,81.333333,16.142857,72.111111,10.300000,8.538462,20.857143,19.872340,99.000000,63.000000,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3936,32959861,True,78,F,57.0,160.0,157.178571,136.833333,38.083333,7.370833,...,65.454545,22.000000,95.785714,8.511111,6.800000,22.000000,16.677419,NaN,NaN,low
3937,32761676,False,75,M,101.0,178.0,179.909091,91.714286,39.000000,7.310000,...,52.142857,17.600000,60.394737,7.250000,5.000000,15.400000,16.795455,NaN,NaN,low
3938,34058245,True,67,F,100.0,165.0,158.331733,104.250000,43.250000,7.292500,...,77.142857,18.800000,122.041667,8.813333,16.700000,30.333333,25.290323,101.789474,59.631579,high
3939,34670930,False,68,M,94.7,163.0,192.500000,90.600000,40.400000,7.414000,...,60.000000,23.500000,65.807692,8.816667,9.166667,21.285714,21.531250,158.000000,83.500000,low


In [4]:
xs = data.copy()

y = xs["mort_28"]
w = xs["peep_regime"]

w = pd.Series(map(lambda x: 0 if x == "low" else 1, w))

xs = xs.drop(columns=["id", "mort_28", "peep", "peep_regime"])

xs.loc[xs["sex"] == "M", "sex"] = 0
xs.loc[xs["sex"] == "F", "sex"] = 1

xs

,age,sex,weight,height,pf_ratio,po2,pco2,ph,driving_pressure,lung_compliance,...,platelets,urea,fio2,hco3,heart_rate,minute_volume,plateau_pressure,respiratory_rate,syst_blood_pressure,diastolic_blood_pressure
0,75,1,93.0,168.0,241.944444,130.666667,28.111111,7.245556,11.000000,31.227273,...,83.000000,3.71280,50.000000,12.000000,115.408163,8.241111,24.333333,18.943662,100.600000,68.200000
1,47,0,127.0,183.0,166.356061,113.076923,34.461538,7.196923,8.350000,79.780702,...,19.600000,32.21925,68.500000,12.750000,115.407407,16.628571,17.500000,19.857143,99.000000,68.000000
2,85,0,77.5,170.0,224.000000,112.000000,38.000000,7.440000,17.250000,33.338235,...,124.500000,16.42200,51.818182,23.000000,70.041667,9.433333,22.333333,15.833333,102.000000,46.333333
3,85,0,90.7,170.0,205.714286,72.000000,38.000000,7.500000,17.000000,26.882353,...,200.000000,24.63300,36.875000,27.000000,70.480000,7.478571,24.000000,18.200000,108.083333,46.208333
4,64,0,70.0,183.0,173.777778,94.666667,38.666667,7.400000,8.585714,52.821226,...,513.666667,5.11700,52.500000,21.666667,73.955556,7.050000,14.328571,15.016667,103.415094,57.377358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3936,78,1,57.0,160.0,157.178571,136.833333,38.083333,7.370833,16.400000,26.829268,...,209.666667,14.63700,65.454545,22.000000,95.785714,8.511111,22.000000,16.677419,NaN,NaN
3937,75,0,101.0,178.0,179.909091,91.714286,39.000000,7.310000,10.400000,42.412137,...,78.200000,28.70280,52.142857,17.600000,60.394737,7.250000,15.400000,16.795455,NaN,NaN
3938,67,1,100.0,165.0,158.331733,104.250000,43.250000,7.292500,13.200000,25.796667,...,309.400000,15.06540,77.142857,18.800000,122.041667,8.813333,30.333333,25.290323,101.789474,59.631579
3939,68,0,94.7,163.0,192.500000,90.600000,40.400000,7.414000,12.500000,33.805421,...,151.200000,6.69375,60.000000,23.500000,65.807692,8.816667,21.285714,21.531250,158.000000,83.500000


In [5]:
y

0       False
1       False
2       False
3       False
4        True
        ...  
3936     True
3937    False
3938     True
3939    False
3940     True
Name: mort_28, Length: 3941, dtype: bool

In [6]:
w

0       0
1       0
2       0
3       0
4       0
       ..
3936    0
3937    0
3938    1
3939    0
3940    0
Length: 3941, dtype: int64